In [ ]:
%pip install h5py

In [ ]:
import librosa
import numpy as np
import os
import glob
from pathlib import Path    
import gc
import h5py

In [ ]:
VOCAL_DIR = '../media_files/separated_audio/train/vocals'
INSTRUMENTAL_DIR = '../media_files/separated_audio/train/instrumentals'
OUTPUT_DIR= '../media_files/preprocessed_audio/train'
OUTPUT_FILE = 'data_optimized.h5'

In [ ]:
# --- Cập nhật tham số mới ---
SR = 44100
N_FFT = 2048
HOP_LENGTH = 512

# Tăng kích thước cửa sổ lên 128 (frames)
WINDOW_SIZE = 128 

# Bước nhảy nên để 50% cửa sổ để tạo độ chồng lấp (overlap), giúp data mượt hơn
WINDOW_STEP = 64

In [ ]:
def match_vocal_and_instrumental(vocal_dir, instrumental_dir):
    matched_pairs = []
    
    # Lấy danh sách file trong thư mục vocal
    vocal_files = sorted(list(Path(vocal_dir).glob("*.wav")))
    
    for v_path in vocal_files:
        # Giả định file instrumental có cùng tên
        i_path = Path(instrumental_dir) / v_path.name
        
        if i_path.exists():
            matched_pairs.append({
                'name': v_path.name,
                'vocal_path': str(v_path),
                'inst_path': str(i_path)
            })
        else:
            print(f"Bỏ qua: Không tìm thấy instrumental cho {v_path.name}")
            
    print(f"Đã tìm thấy {len(matched_pairs)} cặp file hợp lệ.")
    return matched_pairs

matched_data = match_vocal_and_instrumental(VOCAL_DIR, INSTRUMENTAL_DIR)

In [ ]:
def process_single_pair(vocal_path, inst_path):
    """
    Hàm xử lý trọn gói 1 cặp file: Load -> STFT -> Slice -> Trả về X, Y
    """
    try:
        # 1. Load Audio
        y_vocal, _ = librosa.load(vocal_path, sr=SR, mono=True)
        y_inst, _ = librosa.load(inst_path, sr=SR, mono=True)
        
        # Cắt độ dài cho bằng nhau
        min_len = min(len(y_vocal), len(y_inst))
        y_vocal = y_vocal[:min_len]
        y_inst = y_inst[:min_len]
        
        # Tạo Mix
        y_mix = y_vocal + y_inst
        
        # 2. STFT
        mag_vocal = np.abs(librosa.stft(y_vocal, n_fft=N_FFT, hop_length=HOP_LENGTH))
        mag_inst = np.abs(librosa.stft(y_inst, n_fft=N_FFT, hop_length=HOP_LENGTH))
        mag_mix = np.abs(librosa.stft(y_mix, n_fft=N_FFT, hop_length=HOP_LENGTH))
        
        # 3. Power Law Compression (Căn bậc 2) - Tối ưu cho Model
        mag_mix = np.sqrt(mag_mix)
        mag_vocal = np.sqrt(mag_vocal)
        mag_inst = np.sqrt(mag_inst)
        
        # 4. Tạo Mask (Binary Mask)
        # 1 nếu Vocal > Inst, ngược lại 0
        mask = (mag_vocal > mag_inst).astype(np.float32)
        
        # 5. Chuẩn hóa và Cắt (Slicing)
        # Cắt bỏ dòng tần số 1025 -> còn 1024
        mag_mix = mag_mix[:1024, :]
        mask = mask[:1024, :]
        
        # Normalize Mix về 0-1
        max_val = np.max(mag_mix)
        if max_val == 0: max_val = 1
        mag_mix_norm = mag_mix / max_val
        
        # Cắt thành các window nhỏ
        X_list = []
        Y_list = []
        n_frames = mag_mix.shape[1]
        
        for i in range(0, n_frames - WINDOW_SIZE + 1, WINDOW_STEP):
            x_win = mag_mix_norm[:, i:i+WINDOW_SIZE]
            y_win = mask[:, i:i+WINDOW_SIZE]
            
            # Thêm channel dimension: (1024, 128, 1)
            X_list.append(x_win[..., np.newaxis])
            Y_list.append(y_win[..., np.newaxis])
            
        return np.array(X_list, dtype=np.float32), np.array(Y_list, dtype=np.float32)

    except Exception as e:
        print(f"Lỗi khi xử lý {vocal_path}: {e}")
        return np.array([]), np.array([])

In [ ]:
# Xóa file cũ nếu tồn tại
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)
    print(f"Đã xóa file cũ: {OUTPUT_FILE}")

# Mở file H5 để ghi
with h5py.File(OUTPUT_FILE, 'w') as f:
    # Tạo dataset có thể mở rộng (maxshape=None)
    # Shape: (Số mẫu, 1024, 128, 1)
    dset_x = f.create_dataset('X_train', shape=(0, 1024, 128, 1), 
                              maxshape=(None, 1024, 128, 1), 
                              dtype='float32', compression="gzip")
    
    dset_y = f.create_dataset('Y_train', shape=(0, 1024, 128, 1), 
                              maxshape=(None, 1024, 128, 1), 
                              dtype='float32', compression="gzip")
    
    total_samples = 0
    
    # Duyệt qua từng cặp file
    for i, data in enumerate(matched_data):
        print(f"[{i+1}/{len(matched_data)}] Đang xử lý: {data['name']}")
        
        # 1. Xử lý file hiện tại
        X_batch, Y_batch = process_single_pair(data['vocal_path'], data['inst_path'])
        
        # 2. Nếu có dữ liệu, ghi ngay vào đĩa
        if len(X_batch) > 0:
            # Tính toán kích thước mới
            old_size = dset_x.shape[0]
            new_size = old_size + len(X_batch)
            
            # Mở rộng dataset
            dset_x.resize(new_size, axis=0)
            dset_y.resize(new_size, axis=0)
            
            # Ghi dữ liệu vào phần mở rộng
            dset_x[old_size:] = X_batch
            dset_y[old_size:] = Y_batch
            
            total_samples += len(X_batch)
        
        # 3. QUAN TRỌNG: Giải phóng RAM ngay lập tức
        del X_batch, Y_batch
        gc.collect() # Bắt buộc gọi dọn rác bộ nhớ

print(f"\nHOÀN TẤT! Tổng cộng {total_samples} mẫu đã lưu vào {OUTPUT_FILE}")

In [ ]:
# Kiểm tra file đã tạo
with h5py.File(OUTPUT_FILE, 'r') as f:
    print("Thông tin file H5:")
    print("X_train shape:", f['X_train'].shape)
    print("Y_train shape:", f['Y_train'].shape)